In [ ]:
%pip install -Uq upgini catboost

     |████████████████████████████████| 68 kB 4.0 MB/s 
     |████████████████████████████████| 76.6 MB 1.2 MB/s 
     |████████████████████████████████| 1.5 MB 44.6 MB/s 
     |████████████████████████████████| 1.6 MB 58.4 MB/s 
     |████████████████████████████████| 141 kB 54.5 MB/s 


In [ ]:
from os.path import exists
import pandas as pd

df_path = "train.csv.zip" if exists("train.csv.zip") else "https://github.com/Lanxin-Xiang/Store-Item-Demand-Forecasting/blob/main/data/train.csv.zip?raw=true"
df = pd.read_csv(df_path,compression='zip')
# print(df.shape)
# df = df.sample(n=19_000, random_state=0)
df['store'] = df['store'].astype(str)
df['item'] = df['item'].astype(str)
df['date'] = pd.to_datetime(df['date'])
# df.sort_values("date", inplace=True)
# df.reset_index(inplace=True, drop=True)
print(df.shape)
df.head()

In [ ]:
train = df[df['date'] < "2017-01-01"]
test = df[df['date'] >= "2017-01-01"]

In [ ]:
train_features = train.drop(columns='sales')
train_target = train['sales']
test_features = test.drop(columns='sales')
test_target = test['sales']

In [ ]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys = {
        "date": SearchKey.DATE, # email/ip/
    }, 
    cv = CVType.time_series,
    api_key = "kC-ANX-Z3Wth7JNwvGngqEvSFiYDcp3PgGlxC8lwp08"
)
enricher.fit(train_features, train_target, eval_set=[(test_features, test_target)])

<IPython.core.display.Javascript object>

Detected task type: ModelTaskType.REGRESSION


Column name,Status,Description
target,All valid,All values in this column are good to go
date,All valid,All values in this column are good to go


Running search request with search_id=d7a231c7-0645-4fb7-b169-b6e1fa29b776
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done


17 relevant feature(s) found with the search keys: ['date']


,feature_name,shap_value,coverage %,type
0,item,0.471472,100.0,CHARACTER
1,store,0.171505,100.0,CHARACTER
2,f_weather_pca_0_94efd18d,0.133657,100.0,NUMERIC
3,f_week_sin1_a71d22f6,0.044399,100.0,NUMERIC
4,f_year_cos1_cd165f8c,0.025800,100.0,NUMERIC
5,f_week_cos1_d3d56d7f,0.025402,100.0,NUMERIC
6,f_dow_jones_89547e1d,0.007815,100.0,NUMERIC
7,f_cpi_pca_5_db7798a3,0.007120,100.0,NUMERIC
8,f_snp500_61d23988,0.005275,100.0,NUMERIC
9,f_cbpol_umap_1_34dc2149,0.004429,100.0,NUMERIC


In [ ]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)
enricher.calculate_metrics(
    train_features, train_target,
    eval_set=[(test_features, test_target)],
    estimator=model,
    scoring="mean_absolute_percentage_error"
)

Start calculating metrics
Done


,match_rate,baseline mean_absolute_percentage_error,enriched mean_absolute_percentage_error,uplift
,,,,
train,100.0,0.594258,0.550807,0.043451
eval 1,100.0,0.240937,0.536275,-0.295338


In [ ]:
enriched_train_features = enricher.transform(train_features, keep_input=True)
enriched_test_features = enricher.transform(test_features, keep_input=True)
enriched_train_features.head()

99.80000% of the rows are fully duplicated


Column name,Status,Description
date,All valid,All values in this column are good to go


Running search request with search_id=fe5fc92e-749b-496e-be77-b92dbe53d660
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done

Executing transform step
Done
99.80000% of the rows are fully duplicated


Column name,Status,Description
date,All valid,All values in this column are good to go


Running search request with search_id=1096494c-bad8-47e4-8d59-140e82e291db
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done

Executing transform step
Done


,date,store,item,f_weather_pca_0_94efd18d,f_week_sin1_a71d22f6,f_year_cos1_cd165f8c,f_week_cos1_d3d56d7f,f_dow_jones_89547e1d,f_cpi_pca_5_db7798a3,f_snp500_61d23988,f_cbpol_umap_1_34dc2149,f_finance_umap_2_70ce00f3,f_cpi_umap_0_d5602c73,f_weather_umap_30_98fa4f7d,f_cci_pca_1_0ca24b1e,f_cbpol_pca_7_bd0a9516,f_nasdaq_d309709a,f_finance_umap_3_424d51ca,f_payment_fraud_score_3cae9c42,f_cbpol_umap_0_1422bee5
0,2013-01-01,1,1,28.661328,0.781831,0.985220,0.623490,13104.139648,-8.943169,1426.189941,4.815701,10.752728,8.117146,3.547175,12.580468,2.130072,3019.510010,7.647812,0.132107,2.223673
1,2013-01-02,1,1,28.795890,0.974928,0.982126,-0.222521,13104.139648,-8.943169,1426.189941,4.812181,10.734714,8.066880,3.400228,12.580468,2.136201,3019.510010,7.666968,0.110420,2.224823
2,2013-01-03,1,1,28.232902,0.433884,0.978740,-0.900969,13412.549805,-8.943169,1462.420044,4.820669,10.431242,8.096957,3.551585,12.580468,2.097283,3112.260010,7.523365,0.119983,2.220275
3,2013-01-04,1,1,28.041353,-0.433884,0.975065,-0.900969,13391.360352,-8.943169,1459.369995,4.792404,6.774570,8.079817,3.571433,12.580468,2.084673,3100.570068,7.480372,0.113508,2.230913
4,2013-01-05,1,1,29.756092,-0.974928,0.971100,-0.222521,13435.209961,-8.943169,1466.469971,4.784345,10.798021,8.112678,3.597723,12.580468,2.046441,3101.659912,7.465416,0.121708,2.232694


In [ ]:
model.fit(train_features, train_target)
preds = model.predict(test_features)
eval_metric(test_target.values, preds, "SMAPE") # symmetric mean absolute percentage error

[39.511094979842355]

In [ ]:
model.fit(enriched_train_features, train_target)
enriched_preds = model.predict(enriched_test_features)
eval_metric(test_target.values, enriched_preds, "SMAPE") # symmetric mean absolute percentage error

[12.932343760570774]